## knock 039 完全外部結合でデータを残す

In [9]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列が省略されないようにする

polars.config.Config

### データを読み込む

In [10]:
df_receipt = pl.read_csv("../docker/work/data/receipt.csv")
display(df_receipt.head())

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90


### ノック

In [12]:
df_cnt = (
    df_receipt
    # 非会員は除く
    .filter( ~pl.col("customer_id").str.starts_with("Z") )
    # 来店日数を集計するために、来店日時のユニーク値を取り出す
    .unique(subset = ["customer_id", "sales_ymd"])
    # 来店日数を計数する
    .group_by(by = "customer_id")
    .agg( pl.count("sales_ymd"))
    # 上位20名を抽出する
    .sort(by = "sales_ymd", descending = True)
    .head(n = 20)
)

df_sum = (
    df_receipt
    # 非会員は除く
    .filter( ~pl.col("customer_id").str.starts_with("Z") )
    # 合計売上を計算する
    .group_by(by = "customer_id")
    .agg(pl.sum("amount"))
    # 上位20名を抽出する
    .sort(by = "amount", descending = True)
    .head(n = 20)
)

(
    df_cnt
    .join(other = df_sum,
          on = "customer_id",
          how = "outer")
    .sort(by = "sales_ymd", descending = True, nulls_last = True)
)

customer_id,sales_ymd,amount
str,u32,i64
"""CS040214000008…",23,null
"""CS015415000185…",22,20153
"""CS010214000010…",22,18585
"""CS028415000007…",21,19127
"""CS010214000002…",21,null
"""CS017415000097…",20,23086
"""CS016415000141…",20,18372
"""CS031414000051…",19,19202
"""CS021515000172…",19,null
